<a href="https://colab.research.google.com/github/roberth1silva/alura-imersao-ai/blob/main/Roberth_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
%pip install -q google-adk

In [15]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Configura o cliente da SDK do Gemini

from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

# Import Utilidades do ADK
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name="agente_buscador",
      model=MODEL_ID,
      description="Agente que busca informações no google",
      tools=[google_search],
      instruction="""
      Você é uma assitente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
      para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
      Foque em no máximo 5 lançamento relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
      Se um tema tiver poucas notícias ou reações entusiasmadas, é possivel que ele não seja tão relevante assim
      e pode ser substituído por outro que tenha mais.
      Esse lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
      """
  )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  lancamentos = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos

################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model=MODEL_ID,
        description="Agente que planeja posts",
        tools=[google_search],
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é uma planejador de contéudo, especialista em redes sociais. Com base na lista ]
        de lançamentos mais recente e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais informações
        sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevantes entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem
        abordados no post que será escrito posteriormente.
        """
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model=MODEL_ID,
        description="Agente redator de posts engajadores para Instagram",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model=MODEL_ID,
        description="Agente revisor de post para redes sociais.",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
  print(f"Maravilha! vamos então criar o post sobre novidades em {topico}")

  lancamentos = agente_buscador(topico, data_de_hoje)
  print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos))
  print("--------------------------------------------------------------")

  planejamento = agente_planejador(topico, lancamentos)
  print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(planejamento))
  print("--------------------------------------------------------------")

  rascunho = agente_redator(topico, planejamento)
  print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho))
  print("--------------------------------------------------------------")

  revisado = agente_revisor(topico, rascunho)
  print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(revisado))
  print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Uso de celular por jovens
Maravilha! vamos então criar o post sobre novidades em Uso de celular por jovens

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para fornecer as informações mais atuais e relevantes sobre o uso de celulares por jovens, farei algumas buscas no Google para identificar lançamentos, estudos ou tendências recentes que estejam gerando notícias e discussões.
> 
> 
> Com base nas notícias e pesquisas mais recentes, aqui estão 5 lançamentos/tópicos relevantes sobre o uso de celular por jovens, com foco nos últimos meses:
> 
> 1.  **Lei nº 15.100/2025 e restrição do uso de celulares nas escolas:** Aprovada em janeiro de 2025, essa lei restringe o uso de celulares nas escolas brasileiras, visando melhorar a concentração e a saúde mental dos alunos. As escolas estão definindo suas próprias estratégias de implementação, com apoio do Ministério da Educação (MEC) [1, 3, 5, 11, 12, 15, 21, 23, 24]. No entanto, um estudo da Universidade de Birmingham publicado em fevereiro de 2025 aponta que a proibição do uso de celulares nas escolas não melhora o bem-estar mental dos adolescentes, nem o desempenho acadêmico [16].
> 2.  **Impacto do uso excessivo de celulares na saúde mental:** Estudos recentes mostram que o uso excessivo de celulares está associado a ansiedade, depressão, insônia e até pensamentos suicidas em jovens [2, 4, 5, 6, 8, 18, 21, 24]. Uma pesquisa do Instituto Papo de Homem (PDH) revelou que 44% dos jovens admitem ser viciados em seus dispositivos móveis [4, 8]. A Organização Mundial da Saúde (OMS) indica que o transtorno de ansiedade afeta mais de 300 milhões de pessoas no mundo, com um crescimento alarmante entre adolescentes e jovens adultos [18].
> 3.  **Novos recursos em aplicativos para promover o bem-estar digital:** Plataformas como o TikTok lançaram novos recursos para ajudar os jovens a reduzir o tempo de tela e promover o bem-estar digital, como sons relaxantes para a hora de dormir e ferramentas de controle parental [13].
> 4.  **Aplicativos de controle parental:** Há uma variedade de aplicativos disponíveis para ajudar os pais a monitorar e controlar o uso de celulares por seus filhos, como Google Family Link, AppBlock, Controle Parental Screen Time, Life360, GPS Rastreador de Família KidsControl, mSpy, Qustodio e AirDroid Parental Control [9, 17, 19, 20].
> 5.  **Tendências para o público infantojuvenil em 2025:** Uma tendência crescente é a busca por plataformas que ofereçam experiências mais seguras e enriquecedoras, com 86% dos jovens buscando reduzir o tempo de tela. A Geração Alfa está adotando hábitos digitais mais conscientes e valorizando espaços como o YouTube e o Roblox [7].
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Ok, com base nos lançamentos e tópicos que você me forneceu, vou criar um plano de conteúdo para cada um deles e, em seguida, escolher o tema mais relevante para um post detalhado.
> 
> **1. Lei nº 15.100/2025 e restrição do uso de celulares nas escolas:**
> 
> *   **Pontos Relevantes:**
>     *   A lei em si e seus objetivos (melhorar concentração e saúde mental).
>     *   Como as escolas estão implementando a lei (estratégias e apoio do MEC).
>     *   O estudo da Universidade de Birmingham que questiona a eficácia da proibição.
> *   **Plano de Conteúdo:**
>     *   Introdução: Explicar a lei e o contexto.
>     *   Prós e Contras: Apresentar os argumentos a favor da restrição (concentração, saúde mental) e o estudo que a questiona (Universidade de Birmingham).
>     *   Implementação: Como as escolas estão lidando com a lei na prática.
>     *   Discussão: Abrir para debate - a proibição é a melhor solução?
>     *   Enquete: O que os jovens acham da lei?
> 
> **2. Impacto do uso excessivo de celulares na saúde mental:**
> 
> *   **Pontos Relevantes:**
>     *   Associação entre uso excessivo e problemas como ansiedade, depressão, insônia.
>     *   Dados da pesquisa do Instituto Papo de Homem (44% dos jovens se consideram viciados).
>     *   Estatísticas da OMS sobre transtornos de ansiedade em jovens.
> *   **Plano de Conteúdo:**
>     *   Alerta: O impacto negativo do uso excessivo de celulares na saúde mental dos jovens.
>     *   Dados: Apresentar as estatísticas (PDH, OMS).
>     *   Sintomas: Quais são os sinais de que o uso do celular está afetando a saúde mental?
>     *   Dicas: Estratégias para reduzir o tempo de tela e promover o bem-estar.
>     *   Recursos: Indicar links para ajuda profissional e informações adicionais.
> 
> **3. Novos recursos em aplicativos para promover o bem-estar digital:**
> 
> *   **Pontos Relevantes:**
>     *   Exemplos de recursos (sons relaxantes do TikTok, ferramentas de controle parental).
>     *   O objetivo desses recursos (reduzir o tempo de tela e promover o bem-estar).
> *   **Plano de Conteúdo:**
>     *   Novidades: Apresentar os novos recursos de bem-estar digital em aplicativos populares.
>     *   Exemplos: Mostrar como esses recursos funcionam (screenshots, vídeos curtos).
>     *   Opinião: Avaliar a eficácia desses recursos.
>     *   Convite: Incentivar os jovens a experimentar e compartilhar suas experiências.
> 
> **4. Aplicativos de controle parental:**
> 
> *   **Pontos Relevantes:**
>     *   A variedade de aplicativos disponíveis (Google Family Link, Qustodio, etc.).
>     *   As funcionalidades desses aplicativos (monitoramento, controle de tempo, etc.).
> *   **Plano de Conteúdo:**
>     *   Lista: Apresentar os principais aplicativos de controle parental.
>     *   Funcionalidades: Descrever o que cada aplicativo oferece.
>     *   Prós e Contras: Discutir as vantagens e desvantagens do uso desses aplicativos.
>     *   Ética: Levantar questões sobre privacidade e autonomia dos jovens.
>     *   Enquete: Qual a opinião dos jovens sobre o uso de aplicativos de controle parental?
> 
> **5. Tendências para o público infantojuvenil em 2025:**
> 
> *   **Pontos Relevantes:**
>     *   A busca por plataformas mais seguras e enriquecedoras.
>     *   A intenção dos jovens de reduzir o tempo de tela (86%).
>     *   A valorização de espaços como YouTube e Roblox.
> *   **Plano de Conteúdo:**
>     *   Tendências: Apresentar as principais tendências de uso de tecnologia entre jovens.
>     *   Geração Alfa: Descrever os hábitos digitais dessa geração.
>     *   Plataformas: Discutir o que torna YouTube e Roblox tão populares entre os jovens.
>     *   Previsões: O que esperar do futuro do uso de tecnologia por jovens?
> 
> **Escolha do Tema Mais Relevante e Plano Detalhado:**
> 
> Considerando a relevância para a saúde mental dos jovens e o potencial para gerar discussões construtivas, escolho o tema **"Impacto do uso excessivo de celulares na saúde mental"**.
> 
> **Plano Detalhado:**
> 
> *   **Título:** O Preço da Conectividade: Como o Celular Afeta a Saúde Mental dos Jovens
> *   **Introdução (1 parágrafo):**
>     *   Começar com uma estatística impactante (ex: 44% dos jovens se consideram viciados em celular).
>     *   Afirmar que o uso excessivo de celulares está ligado a problemas de saúde mental.
>     *   Indicar que o post irá explorar essa relação e oferecer dicas para um uso mais saudável.
> *   **O Que Dizem os Números (2-3 parágrafos):**
>     *   Apresentar os dados da pesquisa do Instituto Papo de Homem sobre vício em celulares.
>     *   Citar as estatísticas da OMS sobre o aumento de transtornos de ansiedade entre jovens.
>     *   Mencionar outros estudos que relacionam o uso excessivo de celulares com depressão, insônia e pensamentos suicidas.
> *   **Sinais de Alerta (3-4 parágrafos):**
>     *   Quais são os sinais de que o uso do celular está afetando a saúde mental?
>         *   Isolamento social: Preferir ficar online a interagir com amigos e família.
>         *   Ansiedade: Sentir-se ansioso quando está longe do celular.
>         *   Insônia: Dificuldade para dormir devido ao uso do celular antes de dormir.
>         *   Irritabilidade: Ficar irritado ou frustrado quando é interrompido durante o uso do celular.
>         *   Queda no desempenho escolar: Dificuldade para se concentrar nos estudos.
> *   **Dicas para um Uso Mais Saudável (4-5 parágrafos):**
>     *   Definir limites de tempo de tela: Usar aplicativos para monitorar e controlar o tempo gasto no celular.
>     *   Criar horários "livres de tecnologia": Desligar o celular durante as refeições, antes de dormir e em momentos de lazer.
>     *   Encontrar atividades alternativas: Praticar esportes, hobbies, passar tempo com amigos e família.
>     *   Cuidar do sono: Evitar o uso do celular pelo menos uma hora antes de dormir.
>     *   Buscar ajuda profissional: Se sentir que o uso do celular está afetando sua saúde mental, procurar um psicólogo ou terapeuta.
> *   **Conclusão (1-2 parágrafos):**
>     *   Reafirmar a importância de um uso consciente e equilibrado do celular.
>     *   Incentivar os jovens a priorizar sua saúde mental e bem-estar.
>     *   Abrir para discussão: Qual a sua experiência com o uso do celular e a saúde mental? Deixe seu comentário!
> *   **Recursos Adicionais:**
>     *   Links para sites com informações sobre saúde mental (ex: CVV, OMS).
>     *   Indicação de aplicativos para controle de tempo de tela e bem-estar digital.
> *   **Call to Action:**
>     *   Compartilhe este post com seus amigos!
>     *   Siga nossa página para mais dicas sobre saúde mental e bem-estar.
> 
> Este plano detalhado fornece uma estrutura para um post informativo e envolvente sobre o impacto do uso de celulares na saúde mental dos jovens. Ele combina dados, dicas práticas e um convite à discussão, o que pode gerar um bom engajamento nas redes sociais. Antes de escrever o post final, seria útil pesquisar mais a fundo sobre os recursos adicionais (sites, aplicativos) para fornecer indicações mais precisas e relevantes para o público.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> 🚨 Sinal de alerta: será que o celular está te dominando? 📱🤔
> 
> 44% dos jovens se sentem viciados em celular! 🤯 A gente sabe que é difícil largar, mas o uso excessivo pode estar afetando sua saúde mental. 🧠💔
> 
> A ansiedade tem te pegado? 😥 Insônia? 😴 Irritabilidade? 😤 Fique ligado! 👀 Esses podem ser sinais de que você precisa dar um tempo do mundo digital.
> 
> 💡 Mas calma, a Alura te dá um help! 😉
> 
> ✅ Defina limites de tempo de tela
> 📵 Crie horários "detox"
> ⚽ Encontre atividades offline
> 😴 Cuide do seu sono (longe do celular!)
> 🗣️ Se precisar, busque ajuda!
> 
> Lembre-se: sua saúde mental é prioridade! 💚
> 
> 💬 Conta pra gente: como você lida com o uso do celular?
> 
> #SaúdeMental #BemEstarDigital #Alura #TechComPropósito
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos deixá-lo ainda mais interessante e engajador para o público jovem do Instagram. Aqui estão algumas sugestões:
> 
> *   **Emojis:** Usar mais emojis (sem exagerar) pode deixar o post mais divertido e chamativo.
> *   **Chamada para ação:** A pergunta no final ("Conta pra gente: como você lida com o uso do celular?") é boa, mas podemos torná-la mais direta e convidativa.
> *   **Hashtags:** Adicionar algumas hashtags mais específicas pode aumentar o alcance do post.
> 
> **Sugestão de rascunho revisado:**
> 
> 🚨 E aí, addicted em celular? 📱🤔
> 
> Parada obrigatória: 44% da galera se sente viciada no celular! 🤯 A gente sabe que é hard resistir, mas será que o uso excessivo não tá te sabotando? 🧠💔
> 
> Se liga nos sinais:
> 
> *   Crises de ansiedade? 😥
> *   Noites em claro? 😴
> *   Pavio curto? 😤
> 
> Pode ser a hora de dar um tempo do digital! ⏰
> 
> 💡 A Alura te dá o mapa da mina: 😉
> 
> ✅ Metas de tempo de tela
> 📵 Horários de #DetoxDigital
> ⚽ Mais rolê offline, menos online
> 😴 Celular? Só pra despertar!
> 🗣️ Deu ruim? #TerapiaFazBem
> 
> Sua saúde mental é TUDO! 💚
> 
> 💬 E aí, qual sua tática pra equilibrar o online e o offline? 👇 Manda a brasa nos comentários!
> 
> \#SaúdeMental \#BemEstarDigital \#Alura \#TechComPropósito \#VicioEmCelular \#EquilibrioDigital \#OfflineÉONovoONline
> 


--------------------------------------------------------------
